### Importing the Libraries

In [17]:
from flask import Flask, request, jsonify
import nltk
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import random
from nltk.corpus import wordnet
nltk.download('vader_lexicon')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('omw-1.4')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/u188184/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to /home/u188184/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/u188184/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/u188184/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package omw-1.4 to /home/u188184/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [13]:
from sklearnex import patch_sklearn
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer

### Initializing the Analysers

In [2]:
# initialize lemmatizer
lemmatizer = WordNetLemmatizer()

# Initialize Sentiment Intensity Analyzer
sid = SentimentIntensityAnalyzer()

In [20]:
tokenizer = word_tokenize

lemmatizer = WordNetLemmatizer().lemmatize

vectorizer = TfidfVectorizer(tokenizer=tokenizer, stop_words='english', 
                             lowercase=True, use_idf=True, norm='l2', 
                             smooth_idf=True, sublinear_tf=False, 
                             preprocessor=lemmatizer)

### Intializing Keywords and Responses

In [3]:
greetings = ['Hello! I\'m an AI chatbot programmed to assist with mental health concerns. How are you feeling today?', 
             'Hi! I\'m here to help with your mental health concerns. How can I assist you today?', 
             'Welcome! I\'m an AI chatbot designed to support you with your mental health. What\'s been on your mind lately?']
symptom_responses = ['I\'m sorry to hear that you are feeling {}. Can you tell me more about what\'s been causing your {}?', 
                     'It sounds like you are experiencing {}. Would you like to talk more about what\'s been going on?', 
                     'Feeling {} can be tough. Would you like to tell me more about what\'s been happening?']
thought_responses = ['I\'m sorry you are having negative thoughts. Let\'s challenge them. Can you provide me an example of one negative thought you are having?',
                     'I understand you are having pessimistic thoughts. Can you provide me an example?',
                     'I\'m sorry to hear that you feel hopeless. Can you give me an example of a negative thought you\'ve been having?']
behavior_responses = ['I see that you are avoiding certain situations. Let\'s work on this. Can you give me an example of one situation you have been avoiding?',
                      'Procrastination can be tough. Can you tell me more about the tasks you have been putting off?',
                      'It sounds like you have been isolating yourself. Would you like to talk more about why that is?']
technique_responses = ['Great, let\'s start with a thought challenging exercise. Here\'s an example: {}, can you provide me an alternative thought to this?',
                       'Behavioral activation can be helpful. Let\'s set a goal to complete one task today, even if it\'s small. What do you want to accomplish?',
                       'Self-care is important. Let\'s try a relaxation technique such as deep breathing or progressive muscle relaxation. Would you like to try one now?']
unknown_response = ['I\'m sorry, I don\'t understand. Can you please rephrase that?', 
                    'I\'m not sure I understand what you mean. Can you please clarify?', 
                    'I\'m having trouble understanding. Can you please provide more information?']
goodbye_responses = ['Goodbye!', 'Take care.', 'Until next time.']

In [4]:
greeting_keywords = ['hi', 'hello', 'hey', 'hi there', 'hello there']
symptom_keywords = ['anxious', 'anxiety', 'stressed', 'stress', 'depressed', 'depression', 'sad', 'lonely']
thought_keywords = ['negative', 'pessimistic', 'hopeless', 'helpless', 'overthinking', 'rumination']
behavior_keywords = ['avoidance', 'procrastination', 'isolation']
technique_keywords = ['thought challenging', 'behavioral activation', 'relaxation', 'self-care']
goodbye_keywords = ['exit', 'quit', 'goodbye', 'bye']

### Defining Necessary Functions

In [5]:
def get_wordnet_pos(token):
    tag = nltk.pos_tag([token])[0][1][0].upper() #extract the information
    tag_dict = {"J": wordnet.ADJ, #map
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN) #guess noun if unknown

#### Function to process user input

In [6]:
def process_input(user_input):
    # Get sentiment score
    sentiment_score = sid.polarity_scores(user_input)['compound']
    
   # tokenize user input
    tokens = nltk.word_tokenize(user_input.lower())
    
    # lemmatize tokens
    lemmatized_tokens = [lemmatizer.lemmatize(token, get_wordnet_pos(token)) for token in tokens]
    
    # check for keywords
    
    #initialize variables
    greeting = next((keyword for keyword in greeting_keywords if keyword in lemmatized_tokens), False)
    symptom = next((keyword for keyword in symptom_keywords if keyword in lemmatized_tokens), False)
    thought = next((keyword for keyword in thought_keywords if keyword in lemmatized_tokens), False)
    behavior = next((keyword for keyword in behavior_keywords if keyword in lemmatized_tokens), False)
    technique = next((keyword for keyword in technique_keywords if keyword in lemmatized_tokens), False)
    goodbye = next((keyword for keyword in goodbye_keywords if keyword in lemmatized_tokens), False)
 
    # generate a response based on the keywords
    if greeting:
        response = random.choice(greetings)
    elif symptom:
        response = random.choice(symptom_responses).format(symptom)
    elif thought:
        response = random.choice(thought_responses).format(thought)
    elif behavior:
        response = random.choice(behavior_responses).format(behavior)
    elif technique:
        response = random.choice(technique_responses).format(technique)
    elif goodbye:
        response = random.choice(goodbye_responses)
    else:
        response = random.choice(unknown_response)
    
     # reduce dimensions of input data using TSNE
    tsne = TSNE(n_components=2, random_state=42)
    reduced_data = tsne.fit_transform([sentiment_score])
    
    # plot data
    plt.scatter(reduced_data[0][0], reduced_data[0][1], color='blue')
    plt.title('Sentiment Score Visualization')
    plt.xlabel('Dimension 1')
    plt.ylabel('Dimension 2')
    plt.show()
    
    return response

### Flask Deployment

In [7]:
app = Flask(__name__, static_url_path='', static_folder='static')

In [8]:
# Define a route to handle user input and return the sentiment analysis result
@app.route('/chatbot', methods=['POST'])

# Define a function to start the chatbot
def chatbot():

    while True:
        # Get user input
        user_input = request.form['user_input']

        # Check for exit command
        if user_input.lower() in ['exit', 'quit', 'goodbye', 'bye']:
            print("Goodbye!")
            break

        # Generate a response and print it
        response_ = process_input(user_input)
        print(response_)
        return jsonify({'response': response_})

# Define a route to serve the website
@app.route('/')
def index():
    return app.send_static_file('index.html')

In [9]:
if __name__ == '__main__':
    app.run(debug=False)

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


In [10]:
%tb

No traceback available to show.
